<a href="https://colab.research.google.com/github/hyeonseonn/sensing_app/blob/main/0721_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense


In [ ]:

# ============================================
# 1. 샘플 CPU 데이터 생성 (사인파 + 노이즈)
# ============================================
np.random.seed(42)
minutes = 500   # 500분(약 8시간)
time_index = pd.date_range('2024-01-01', periods=minutes, freq='Min')

# CPU 사용률 패턴: 주기적 + 노이즈
cpu = 50 + 30*np.sin(np.linspace(0, 10, minutes)) + np.random.normal(0, 5, minutes)

df = pd.DataFrame({'timestamp': time_index, 'cpu': cpu})
df.set_index('timestamp', inplace=True)

# ============================================
# 2. 정규화
# ============================================
scaler = MinMaxScaler()
scaled_cpu = scaler.fit_transform(df[['cpu']].values)

# ============================================
# 3. 시퀀스 생성 함수
# ============================================
def create_sequences(dataset, past_steps=20, future_steps=5):
    X, y = [], []
    for i in range(len(dataset)-past_steps-future_steps):
        X.append(dataset[i:i+past_steps])          # 과거 20분
        y.append(dataset[i+past_steps:i+past_steps+future_steps])  # 미래 5분
    return np.array(X), np.array(y)

past_steps = 20
future_steps = 5
X, y = create_sequences(scaled_cpu, past_steps, future_steps)

# ============================================
# 4. 학습/테스트 분리
# ============================================
train_size = int(len(X)*0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

print(f"입력 X_train shape: {X_train.shape}, 출력 y_train shape: {y_train.shape}")

# ============================================
# 5. CNN-LSTM 모델 정의
# ============================================
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu',
                 input_shape=(X_train.shape[1], X_train.shape[2])))  # (time_steps, 1변수)
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(64, activation='relu'))
model.add(Dense(future_steps))  # 미래 5분 CPU 예측
model.compile(optimizer='adam', loss='mse')

# ============================================
# 6. 학습
# ============================================
model.fit(X_train, y_train.reshape((y_train.shape[0], -1)),
          epochs=50, batch_size=16, verbose=1)

# ============================================
# 7. 예측
# ============================================
pred = model.predict(X_test)
pred = pred.reshape(-1, future_steps)  # 미래 5분 예측
y_true = y_test.reshape(-1, future_steps)

# ============================================
# 8. 시각화 (미래 1분 후 예측만 비교)
# ============================================
plt.figure(figsize=(12,5))
plt.plot(y_true[:100,0], label="실제 CPU(1분 후)")
plt.plot(pred[:100,0], label="예측 CPU(1분 후)")
plt.legend()
plt.title("CPU 사용률 CNN-LSTM 예측 (단변수)")
plt.show()
